In [6]:
import cvxpy as cp #conseillé par l'énoncé
import numpy as np #vecteurs et matrices
import matplotlib.pyplot as plt #pour les graphes
from scipy.optimize import _highs #solver conseillé
import math #pour les tanh mais apparemment numpy les import déjà

#on va écrire le problème sous la forme "min c^T x tel que Ax >= b et x >= 0" où c est le vecteur coût et x = x_t + y_t
#avec x_t la puissance par la pompe en temps normal et y_t la puissance par la pompe en temps reverse

#différents paramètres
nb_intervalles = 7244
temperatures_ext = np.load("Temperatures-Montreal.npy")
eta = 0.99
capacite_calorifique = 0.4 * 360 #en kWh
T_min = 19
T_max = 21
T_int = cp.Variable(nb_intervalles)
cost_electricity = np.array([0.18 if (i * 4) % 24 < 7 or (i * 4) % 24 >= 22 else 0.26 for i in range(nb_intervalles)]) #coût qui est variable
P_chauff = cp.Variable(nb_intervalles) #puissance par la pompe en temps normal
P_refroid = cp.Variable(nb_intervalles)#puissance par la pompe en mode inverse

contraintes = [] #matrice A
#début de la résolution, il ne faut pas oubleir que T_0 = T_672 qui doivent etre entre T_min et T_max
for i in range(nb_intervalles):
    contraintes.append(T_int[i] >= T_min)
    contraintes.append(T_int[i] <= T_max)

for i in range(nb_intervalles - 1):
    COP_chauff = 3 + 10 * np.abs(np.tanh(temperatures_ext[i] / 100)) * np.tanh(temperatures_ext[i] / 100)
    COP_refroid = 3.2  # valeur constante pour COP_refroid

    contraintes.append(T_int[i + 1] - T_int[i] - (1 - eta) * (temperatures_ext[i] - T_int[i]))

#On finit les contraintes
contraintes.append(P_chauff >= 0) #puissances positives
contraintes.append(P_refroid >= 0)

cost = cp.sum(cost_electricity * (P_chauff + P_refroid)) #coût total
res = cp.Problem(cp.Minimize(cost), contraintes)

#res = _highs.(cost, contraintes_ub=contraintes, b_ub=b, options={"disp": False}) #ici je sais pas trop comment faire

[ -0.83  -0.83  -0.83 ... -11.1  -11.1  -11.1 ]


/home/iss25/.local/lib/python3.10/site-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 5 times so far.

  warnings.warn(msg, UserWarning)


ValueError: Problem has an invalid constraint of type <class 'cvxpy.atoms.affine.add_expr.AddExpression'>